In [474]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import OneHotEncoder

# modeling
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import *
from sklearn.linear_model import LogisticRegression

#explainability
import shap

#serialization
import joblib

In [475]:
#read the data 
app_train = pd.read_csv("../raw_data/application_train.csv").astype("object")

# Features Types

We need to convert variables that are not correctly cast as of float in the raw data : 

In [476]:
def convert_dtypes_func(df):
    #List the columns to be converted to float
    list_columns_to_convert_to_float = ("CNT_CHILDREN" 
                                ,"AMT_INCOME_TOTAL" 
                                ,"AMT_CREDIT" 
                                ,"AMT_ANNUITY" 
                                ,"AMT_GOODS_PRICE" 
                                ,"DAYS_BIRTH"
                                ,"DAYS_EMPLOYED" 
                                ,"DAYS_REGISTRATION" 
                                ,"DAYS_ID_PUBLISH"
                                ,"OWN_CAR_AGE"
                                ,"EXT_SOURCE_1               "
                                ,"EXT_SOURCE_2               "
                                ,"EXT_SOURCE_3               "
                                ,"APARTMENTS_AVG             "
                                ,"BASEMENTAREA_AVG           "
                                ,"YEARS_BEGINEXPLUATATION_AVG"
                                ,"YEARS_BUILD_AVG            "
                                ,"COMMONAREA_AVG             "
                                ,"ELEVATORS_AVG              "
                                ,"ENTRANCES_AVG              "
                                ,"FLOORSMAX_AVG              "
                                ,"FLOORSMIN_AVG              "
                                ,"LANDAREA_AVG               "
                                ,"LIVINGAPARTMENTS_AVG       "
                                ,"LIVINGAREA_AVG             "
                                ,"NONLIVINGAPARTMENTS_AVG    "
                                ,"NONLIVINGAREA_AVG          "
                                ,"APARTMENTS_MODE            "
                                ,"BASEMENTAREA_MODE          "
                                ,"YEARS_BEGINEXPLUATATION_MODE"
                                ,"YEARS_BUILD_MODE           "
                                ,"COMMONAREA_MODE            "
                                ,"ELEVATORS_MODE             "
                                ,"ENTRANCES_MODE             "
                                ,"FLOORSMAX_MODE             "
                                ,"FLOORSMIN_MODE             "
                                ,"LANDAREA_MODE              "
                                ,"LIVINGAPARTMENTS_MODE      "
                                ,"LIVINGAREA_MODE            "
                                ,"NONLIVINGAPARTMENTS_MODE   "
                                ,"NONLIVINGAREA_MODE         "
                                ,"APARTMENTS_MEDI            "
                                ,"BASEMENTAREA_MEDI          "
                                ,"YEARS_BEGINEXPLUATATION_MEDI"
                                ,"YEARS_BUILD_MEDI           "
                                ,"COMMONAREA_MEDI            "
                                ,"ELEVATORS_MEDI             "
                                ,"ENTRANCES_MEDI             "
                                ,"FLOORSMAX_MEDI             "
                                ,"FLOORSMIN_MEDI             "
                                ,"LANDAREA_MEDI              "
                                ,"LIVINGAPARTMENTS_MEDI      "
                                ,"LIVINGAREA_MEDI            "
                                ,"NONLIVINGAPARTMENTS_MEDI   "
                                ,"NONLIVINGAREA_MEDI         "
                                ,"TOTALAREA_MODE             "
                                ,"OBS_30_CNT_SOCIAL_CIRCLE   "
                                ,"DEF_30_CNT_SOCIAL_CIRCLE   "
                                ,"OBS_60_CNT_SOCIAL_CIRCLE   "
                                ,"DEF_60_CNT_SOCIAL_CIRCLE   "
                                ,"DAYS_LAST_PHONE_CHANGE     "
                                ,"AMT_REQ_CREDIT_BUREAU_HOUR"
                                ,"AMT_REQ_CREDIT_BUREAU_DAY "
                                ,"AMT_REQ_CREDIT_BUREAU_WEEK"
                                ,"AMT_REQ_CREDIT_BUREAU_MON "
                                ,"AMT_REQ_CREDIT_BUREAU_QRT "
                                ,"AMT_REQ_CREDIT_BUREAU_YEAR")       

    #remove spaces in the list created
    list_columns_to_convert_to_float = [s.strip() for s in list_columns_to_convert_to_float]       
    
    convert_count = 0

    #convert object columns to float
    for col in list_columns_to_convert_to_float:
        df[col] = df[col].astype(float)
        
        # Keep track of how many columns were label encoded
        convert_count += 1

    print('%d object columns were converted to int.' % convert_count)
    
    return df

In [477]:
app_train = convert_dtypes_func(app_train)

67 object columns were converted to int.


# Feature Engineering

In [478]:
def feature_eng(df):

    # Create an anomalous flag column
    df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243

    # Replace the anomalous values with nan
    df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

    #correct sign of Days Birth
    df["DAYS_BIRTH"] = abs(df["DAYS_BIRTH"])
    df['AGE_INT'] = round((df['DAYS_BIRTH'] / 365).astype(float),2)

    #footing financial ratios
    df['annuity_income_ratio'] = round((df['AMT_INCOME_TOTAL'] / df['AMT_ANNUITY']).astype(float),2)
    df['credit_annuity_ratio'] = round((df['AMT_CREDIT'] / df['AMT_ANNUITY']).astype(float),2)
    df['credit_goods_price_ratio'] = round((df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']).astype(float),2)
    df['credit_downpayment'] = round((df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']).astype(float),2)
 
    print('Feature engineering success')
   
    return df

In [479]:
app_train = feature_eng(app_train)

Feature engineering success


# Feature Selection

In [480]:
def df_feature_selection(df):
    columns_list = df.columns.to_list()

    columns_to_drop_intersec = (['DAYS_BIRTH',
    'REGION_RATING_CLIENT_W_CITY',
    'BASEMENTAREA_MODE',
    'YEARS_BUILD_MODE',
    'COMMONAREA_MODE',
    'ELEVATORS_MODE',
    'ENTRANCES_MODE',
    'FLOORSMAX_MODE',
    'LANDAREA_MODE',
    'LIVINGAPARTMENTS_MODE',
    'NONLIVINGAPARTMENTS_MODE',
    'NONLIVINGAREA_MODE',
    'APARTMENTS_MEDI',
    'BASEMENTAREA_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI',
    'COMMONAREA_MEDI',
    'ELEVATORS_MEDI',
    'ENTRANCES_MEDI',
    'FLOORSMAX_MEDI',
    'FLOORSMIN_MEDI',
    'LIVINGAPARTMENTS_MEDI',
    'LIVINGAREA_MEDI',
    'NONLIVINGAPARTMENTS_MEDI',
    'NONLIVINGAREA_MEDI',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    "DAYS_EMPLOYED_ANOM"])

    #Drop columns
    df_selec_col = pd.DataFrame(data=df, columns=columns_list).drop(columns_to_drop_intersec, axis=1)
    
    print('Feature selection success')

    return df_selec_col

In [481]:
app_train = df_feature_selection(app_train)

Feature selection success


# Imputation - Encoding - Standardization - Imputation

In [482]:
#keep id into a separate serie
user_id = app_train[['SK_ID_CURR']]

#create df of features by type
cat_features = app_train.select_dtypes(include=['object']).drop(['SK_ID_CURR','TARGET'], axis=1)
num_features = app_train.select_dtypes(exclude=['object'])

#imputation
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
cat_features = categorical_imputer.fit_transform(cat_features)

simple_imputer = SimpleImputer(missing_values=np.nan, strategy="median")
num_features = simple_imputer.fit_transform(num_features)

#One hot encoding categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')
cat_array = ohe.fit_transform(cat_features).todense()
cat_array = np.asarray(cat_array)

#Standard Scaling numerical variables
scaler = StandardScaler()
num_array = scaler.fit_transform(num_features)

#concatenate
X = np.concatenate([cat_array, num_array], axis=1)
y = app_train['TARGET'].astype(int)

In [483]:
#Model with best params
model = LogisticRegression(class_weight = 'balanced', C=0.005, max_iter=1000)

#fit
model.fit(X,y)

LogisticRegression(C=0.005, class_weight='balanced', max_iter=1000)

# Serialization

In [484]:
joblib.dump(ohe, 'bin/ohe.joblib') #into a folder bin (for binary)
joblib.dump(categorical_imputer, 'bin/categorical_imputer.joblib')
joblib.dump(simple_imputer, 'bin/simple_imputer.joblib')
joblib.dump(scaler, 'bin/scaler.joblib')
joblib.dump(model, 'bin/model.joblib')


['bin/model.joblib']

# Export DataSet

Export pre-processed dataset to be used for Dashboard Streamlit (on which heroku deployed model will predicting scoring) : 

In [485]:
app_test = pd.read_csv("../raw_data/application_test.csv").astype("object")

app_test_converted = convert_dtypes_func(app_test)
app_test_eng = feature_eng(app_test)
app_test_selec_col = df_feature_selection(app_test_eng)

df_test = app_test_selec_col.sample(frac=0.05, random_state=0)
df_test = df_test.astype('object')

df_test.to_csv("./dashboard_data/df_test.csv", index=False)

print('df_test exported to dashboard_data folder.')


67 object columns were converted to int.
Feature engineering success
Feature selection success
df_test exported to dashboard_data folder.


# Pydantic Documentation

In [486]:
from pydantic import BaseModel, create_model, Field, ValidationError

model = {}
for i in range(len(app_train.dtypes)):
    name = app_train.dtypes.index[i]
    var_type = type(app_train.iloc[0, i])
    model.update({name: (var_type, Field(...))})

In [487]:
del model["SK_ID_CURR"]
del model["TARGET"]
del model["DAYS_EMPLOYED_ANOM"]

In [488]:
data_dict = joblib.dump(model, 'bin/data_dict.joblib')

In [39]:
colonnes_scoring_predicted = ["CODE_GENDER",
                              "NAME_EDUCATION_TYPE",
                              "DAYS_EMPLOYED"
                              "EXT_SOURCE_1",
                              "EXT_SOURCE_2",
                              "EXT_SOURCE_3",
                              "credit_downpayment",
                              "AMT_INCOME_TOTAL",
                              "annuity_income_ratio",
                              ]

In [40]:
liste = [i for i in df_test.columns.to_list()]
matching = [s for s in liste if "AGE" in s]
matching

['OWN_CAR_AGE', 'AGE_INT']

In [41]:
liste_value = df_test['CODE_GENDER'].value_counts().tolist()
liste_value

[1613, 824]

In [ ]:
df_test = pd.read_csv("./dashboard_data/df_test.csv")

In [358]:
list_cat_features = ["NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "REGION_POPULATION_RELATIVE",
    "FLAG_MOBIL",
    "FLAG_EMP_PHONE",
    "FLAG_WORK_PHONE",
    "FLAG_CONT_MOBILE",
    "FLAG_PHONE",
    "FLAG_EMAIL",
    "OCCUPATION_TYPE",
    "CNT_FAM_MEMBERS",
    "REGION_RATING_CLIENT",
    "WEEKDAY_APPR_PROCESS_START",
    "HOUR_APPR_PROCESS_START",
    "REG_REGION_NOT_LIVE_REGION",
    "REG_REGION_NOT_WORK_REGION",
    "LIVE_REGION_NOT_WORK_REGION",
    "REG_CITY_NOT_LIVE_CITY",
    "REG_CITY_NOT_WORK_CITY",
    "LIVE_CITY_NOT_WORK_CITY",
    "ORGANIZATION_TYPE",
    "FONDKAPREMONT_MODE",
    "HOUSETYPE_MODE",
    "WALLSMATERIAL_MODE",
    "EMERGENCYSTATE_MODE",
    "FLAG_DOCUMENT_2",
    "FLAG_DOCUMENT_3",
    "FLAG_DOCUMENT_4",
    "FLAG_DOCUMENT_5",
    "FLAG_DOCUMENT_6",
    "FLAG_DOCUMENT_7",
    "FLAG_DOCUMENT_8",
    "FLAG_DOCUMENT_9",
    "FLAG_DOCUMENT_10",
    "FLAG_DOCUMENT_11",
    "FLAG_DOCUMENT_12",
    "FLAG_DOCUMENT_13",
    "FLAG_DOCUMENT_14",
    "FLAG_DOCUMENT_15",
    "FLAG_DOCUMENT_16",
    "FLAG_DOCUMENT_17",
    "FLAG_DOCUMENT_18",
    "FLAG_DOCUMENT_19",
    "FLAG_DOCUMENT_20",
    "FLAG_DOCUMENT_21"
]
list_num_features = [
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "DAYS_EMPLOYED",
    "DAYS_REGISTRATION",
    "DAYS_ID_PUBLISH",
    "OWN_CAR_AGE",
    "EXT_SOURCE_1",
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
    "APARTMENTS_AVG",
    "BASEMENTAREA_AVG",
    "YEARS_BEGINEXPLUATATION_AVG",
    "YEARS_BUILD_AVG",
    "COMMONAREA_AVG",
    "ELEVATORS_AVG",
    "ENTRANCES_AVG",
    "FLOORSMAX_AVG",
    "FLOORSMIN_AVG",
    "LANDAREA_AVG",
    "LIVINGAPARTMENTS_AVG",
    "LIVINGAREA_AVG",
    "NONLIVINGAPARTMENTS_AVG",
    "NONLIVINGAREA_AVG",
    "APARTMENTS_MODE",
    "YEARS_BEGINEXPLUATATION_MODE",
    "FLOORSMIN_MODE",
    "LIVINGAREA_MODE",
    "LANDAREA_MEDI",
    "TOTALAREA_MODE",
    "OBS_30_CNT_SOCIAL_CIRCLE",
    "DEF_30_CNT_SOCIAL_CIRCLE",
    "DEF_60_CNT_SOCIAL_CIRCLE",
    "DAYS_LAST_PHONE_CHANGE",
    "AMT_REQ_CREDIT_BUREAU_HOUR",
    "AMT_REQ_CREDIT_BUREAU_DAY",
    "AMT_REQ_CREDIT_BUREAU_WEEK",
    "AMT_REQ_CREDIT_BUREAU_MON",
    "AMT_REQ_CREDIT_BUREAU_QRT",
    "AMT_REQ_CREDIT_BUREAU_YEAR",
    "AGE_INT",
    "annuity_income_ratio",
    "credit_annuity_ratio",
    "credit_goods_price_ratio",
    "credit_downpayment"
]


In [489]:
from pydantic import BaseModel, create_model

data_dict = joblib.load("./bin/data_dict.joblib")
ohe = joblib.load("./bin/ohe.joblib")
categorical_imputer = joblib.load("./bin/categorical_imputer.joblib")
simple_imputer = joblib.load("./bin/simple_imputer.joblib")
scaler = joblib.load("./bin/scaler.joblib")
model = joblib.load("./bin/model.joblib")


In [495]:
type(data_dict)

dict

In [490]:
ScoringModel = create_model(
    "ScoringModel",
    **data_dict,
    __base__=BaseModel,
)

In [497]:
print(ScoringModel.dict())

TypeError: dict() takes exactly 1 positional argument (0 given)

In [491]:
ScoringModel.update_forward_refs()

item_dict = ScoringModel.dict()

df = pd.DataFrame(data=[item_dict.values()], columns=item_dict.keys())
df = df.astype(object)

cat_array = categorical_imputer.transform(df[list_cat_features])
cat_array = ohe.transform(cat_array).todense()

num_array = simple_imputer.transform(df[list_num_features])
# num_array = df[list_num_features].to_numpy()
num_array = scaler.transform(num_array)

X = np.concatenate[cat_array, num_array]
X = np.asarray(X)

TypeError: dict() takes exactly 1 positional argument (0 given)